In [2]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [9]:
pip install tensorflow

You should consider upgrading via the 'C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [10]:
from konlpy.tag import Komoran


# 토크나이즈

In [11]:
class Preprocess:
    def __init__(self, servicedic=None): # userdic 인자에는 사용자 정의 사전 파일 경로 입력가능

        # 형태소 분석기 초기화
        self.komoran = Komoran(userdic=servicedic)

        # 제외할 품사
        # 참조 : https://docs.komoran.kr/firststep/postypes.html
        # 관계언 제거, 기호 제거
        # 어미 제거
        # 접미사 제거
        self.exclusion_tags = [
            'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
            # 주격조사, 보격조사, 관형격조사, 목적격조사, 부사격조사, 호격조사, 인용격조사
            'JX', 'JC',
            # 보조사, 접속조사
            'SF', 'SP', 'SS', 'SE', 'SO',
            # 마침표,물음표,느낌표(SF), 쉼표,가운뎃점,콜론,빗금(SP), 따옴표,괄호표,줄표(SS), 줄임표(SE), 붙임표(물결,숨김,빠짐)(SO)
            'EP', 'EF', 'EC', 'ETN', 'ETM',
            # 선어말어미, 종결어미, 연결어미, 명사형전성어미, 관형형전성어미
            'XSN', 'XSV', 'XSA'
            # 명사파생접미사, 동사파생접미사, 형용사파생접미사
        ]


    # 형태소 분석기 POS 태거
    def pos(self, sentence):
        return self.komoran.pos(sentence)

    # 불용어 제거 후 필요한 품사 정보만 가져오기
    def get_keywords(self, pos, without_tag=False):
        f = lambda x: x in self.exclusion_tags
        word_list = []
        for p in pos:
            if f(p[1]) is False:
                word_list.append(p if without_tag is False else p[0])
        return word_list

## TEST

In [12]:
sent = "구희영 보고싶어"

# 전처리 객체 생성
p = Preprocess(servicedic='./files/service_dic.tsv')

# 형태소 분석기 실행
pos = p.pos(sent)

# 품사 태그와 같이 키워드 출력
ret = p.get_keywords(pos, without_tag=False)
print(ret)

# 품사 태그 없이 키워드 출력
ret = p.get_keywords(pos, without_tag=True)
print(ret)

[('구희영', 'NNP'), ('보', 'VV'), ('싶', 'VX')]
['구희영', '보', '싶']


# 단어 사전

In [13]:
# 단어 사전 파일 생성 코드입니다.
# 챗봇에 사용하는 사전 파일

from tensorflow.keras import preprocessing
import pickle
import pandas as pd

# 말뭉치 데이터 읽어오기
movie_review = pd.read_csv('data/영화리뷰.csv')
purpose = pd.read_csv('data/용도별목적대화데이터.csv')
topic = pd.read_csv('data/주제별일상대화데이터.csv')
common_sense = pd.read_csv('data/일반상식.csv')

movie_review.dropna(inplace=True)
purpose.dropna(inplace=True)
topic.dropna(inplace=True)
common_sense.dropna(inplace=True)

text1 = list(movie_review['document'])
text2 = list(purpose['text'])
text3 = list(topic['text'])
text4 = list(common_sense['query']) + list(common_sense['answer'])

corpus_data = text1 + text2 + text3 + text4

# 말뭉치 데이터에서 키워드만 추출해서 사전 리스트 생성
p = Preprocess()
dict = []
for c in corpus_data:
    pos = p.pos(c)
    for k in pos:
        dict.append(k[0])

# 사전에 사용될 word2index 생성
# 사전의 첫 번째 인덱스에는 OOV 사용
tokenizer = preprocessing.text.Tokenizer(oov_token='OOV', num_words=100000)
tokenizer.fit_on_texts(dict)
word_index = tokenizer.word_index
print(len(word_index))

# 사전 파일 생성
f = open("chatbot_dict.bin", "wb")
try:
    pickle.dump(word_index, f)
except Exception as e:
    print(e)
finally:
    f.close()

KeyboardInterrupt: 

## TEST ) 단어 사전

In [14]:
# 단어 사전 불러오기
f = open("chatbot_dict.bin", "rb")
word_index = pickle.load(f)
f.close()

sent = "구희영 언제 만나더라?"

# 전처리 객체 생성
p = Preprocess(servicedic='files/service_dic.tsv')

# 형태소 분석기 실행
pos = p.pos(sent)

# 품사 태그 없이 키워드 출력
keywords = p.get_keywords(pos, without_tag=True)
for word in keywords:
    try:
        print(word, word_index[word])
    except KeyError:
        # 해당 단어가 사전에 없는 경우 OOV 처리
        print(word, word_index['OOV'])

구희영 1
언제 162
만나 359


# 텍스트 전처리
- 토크 나이즈
- 생성한 단어 사전으로 단어를 정수 인코딩

In [15]:
class Preprocess:
    def __init__(self, word2index_dic='chatbot_dict.bin', servicedic=None): # userdic 인자에는 사용자 정의 사전 파일 경로 입력가능
        
        if (word2index_dic != ''):
            f = open(word2index_dic, 'rb')
            self.word_index = pickle.load(f)
            f.close()
            print('word_dic 로드 성공')
        else:
            self.word_index = None
            print('word_dic 로드 실패')
        
        # 형태소 분석기 초기화
        self.komoran = Komoran(userdic=servicedic)

        # 제외할 품사
        # 참조 : https://docs.komoran.kr/firststep/postypes.html
        # 관계언 제거, 기호 제거
        # 어미 제거
        # 접미사 제거
        self.exclusion_tags = [
            'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
            # 주격조사, 보격조사, 관형격조사, 목적격조사, 부사격조사, 호격조사, 인용격조사
            'JX', 'JC',
            # 보조사, 접속조사
            'SF', 'SP', 'SS', 'SE', 'SO',
            # 마침표,물음표,느낌표(SF), 쉼표,가운뎃점,콜론,빗금(SP), 따옴표,괄호표,줄표(SS), 줄임표(SE), 붙임표(물결,숨김,빠짐)(SO)
            'EP', 'EF', 'EC', 'ETN', 'ETM',
            # 선어말어미, 종결어미, 연결어미, 명사형전성어미, 관형형전성어미
            'XSN', 'XSV', 'XSA'
            # 명사파생접미사, 동사파생접미사, 형용사파생접미사
        ]


    # 형태소 분석기 POS 태거
    def pos(self, sentence):
        return self.komoran.pos(sentence)

    # 불용어 제거 후 필요한 품사 정보만 가져오기
    def get_keywords(self, pos, without_tag=False):
        f = lambda x: x in self.exclusion_tags
        word_list = []
        for p in pos:
            if f(p[1]) is False:
                word_list.append(p if without_tag is False else p[0])
        return word_list
    
    
    def get_wordidx_sequence(self, keywords):
        if self.word_index is None:
            return []
        
        w2i = []
        for word in keywords:
            try:
                w2i.append(self.word_index[word])
            except KeyError:
                w2i.append(self.word_index['OOV'])
        return w2i

## TEST ) 텍스트 전처리

In [17]:
sent = "구희영 언제 만나"

# 전처리 객체 생성
p = Preprocess(servicedic='./files/service_dic.tsv')

# 형태소 분석기 실행
pos = p.pos(sent)

# 품사 태그와 같이 키워드 출력
ret = p.get_keywords(pos, without_tag=False)
print(ret)

# 품사 태그 없이 키워드 출력
ret = p.get_keywords(pos, without_tag=True)
print(ret)

# 단어 임베딩 리스트 출력
embedded_lst = p.get_wordidx_sequence(keywords=sent)
print(embedded_lst)

word_dic 로드 성공
[('구희영', 'NNP'), ('언제', 'MAG'), ('만나', 'VV')]
['구희영', '언제', '만나']
[422, 3885, 1042, 1, 3817, 287, 1, 56, 13]


# 의도 분류 모델
- import
- 데이터 불러오기
- 전처리
- 데이터 변환 (텐서화, 학습/validation/test 데이터 나눔
- 하이퍼 파라미터 설정
- CNN 모델 정의
- CNN 모델 생성
- train
- validation
- test
- 모델 저장

In [18]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [27]:
data = pd.read_csv("data/train_data.csv")
text = data['text'].tolist()
label = data['label'].tolist()

In [29]:
p = Preprocess(word2index_dic='chatbot_dict.bin',
               servicedic='files/service_dic.tsv')

# Data preprocess
sequences = []
for sentence in text:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

word_dic 로드 성공


In [35]:
# set padding length & pad to sequences
from config.GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# data to tensor
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, label))
ds = ds.shuffle(len(text))

# set train & validation & test size
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(100)
val_ds = ds.take(train_size).take(val_size).batch(100)
test_ds = ds.take(train_size + val_size).take(test_size).batch(100)

In [36]:
# Hyperparameter
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 3
VOCAB_SIZE = len(p.word_index) + 1

In [51]:
# CNN model definition
input_layer = Input(shape=(MAX_SEQ_LEN, ))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)

logits = Dense(4, name='logits')(dropout_hidden)
predictions = Dense(4, activation=tf.nn.softmax)(logits)

In [52]:
# CNN model create
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [53]:
# train model
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/3
170/170 [==============================] - 37s 210ms/step - loss: 0.3709 - accuracy: 0.8523 - val_loss: 0.0287 - val_accuracy: 0.9918
Epoch 2/3
170/170 [==============================] - 34s 202ms/step - loss: 0.0324 - accuracy: 0.9909 - val_loss: 0.0085 - val_accuracy: 0.9975
Epoch 3/3
170/170 [==============================] - 36s 213ms/step - loss: 0.0153 - accuracy: 0.9956 - val_loss: 0.0101 - val_accuracy: 0.9971


In [55]:
# evaluate model
loss, accuracy = model.evaluate(test_ds, verbose=1)
print("Accuracy: %f" % (accuracy * 100))
print("loss : %f" % (loss))

25/25 [==============================] - 1s 19ms/step - loss: 0.0042 - accuracy: 0.9979
Accuracy: 99.793816
loss : 0.004231


In [56]:
# save model
model.save('intent_model.h5')

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# TEST ) 의도 분류 모델

In [57]:
p = Preprocess(word2index_dic='chatbot_dict.bin',
               servicedic='files/service_dic.tsv')

word_dic 로드 성공


In [62]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing
import gc

# 의도 분류 모델 모듈
class IntentModel:
    def __init__(self, model_name, preprocess):

        # 의도 클래스별 레이블블
        self.labels = {0: "장소", 1: "시간", 2: "사람", 3 : "내용"}

        # 의도 분류 모델 불러오기
        self.model = load_model(model_name)

        # 챗봇 텍스트 전처리기
        self.p = preprocess

    # 의도 클래스 예측
    def predict_class(self, query):
        # 형태소 분석
        pos = self.p.pos(query)

        # 문장내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 단어 시퀀스 벡터 크기
        from config.GlobalParams import MAX_SEQ_LEN

        # 패딩처리
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

        predict = self.model.predict(padded_seqs)
        predict_class = tf.math.argmax(predict, axis=1)
        return predict_class.numpy()[0]

In [63]:
intent = IntentModel(model_name='intent_model.h5', preprocess=p)

In [64]:
query = "토요일 약속 장소 어디지?"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print("="*30)
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)


query = "언제 구희영 만나더라?"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print("="*30)
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)


query = "13일 약속 누구누구 가"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print("="*30)
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)


query = "부산대 누구랑 가더라?"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print("="*30)
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)


query = "구희영 보고싶어"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print("="*30)
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)


query = "디띵방 투표 기간 언제까지야?"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print("="*30)
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

query = "일요일에 뭐하기로 했지"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print("="*30)
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)
print("="*30)

1/1 [==============================] - 0s 92ms/step
토요일 약속 장소 어디지?
의도 예측 클래스 :  0
의도 예측 레이블 :  장소
1/1 [==============================] - 0s 20ms/step
언제 구희영 만나더라?
의도 예측 클래스 :  1
의도 예측 레이블 :  시간
1/1 [==============================] - 0s 18ms/step
13일 약속 누구누구 가
의도 예측 클래스 :  2
의도 예측 레이블 :  사람
1/1 [==============================] - 0s 18ms/step
부산대 누구랑 가더라?
의도 예측 클래스 :  2
의도 예측 레이블 :  사람
1/1 [==============================] - 0s 19ms/step
구희영 보고싶어
의도 예측 클래스 :  1
의도 예측 레이블 :  시간
1/1 [==============================] - 0s 19ms/step
디띵방 투표 기간 언제까지야?
의도 예측 클래스 :  1
의도 예측 레이블 :  시간
1/1 [==============================] - 0s 23ms/step
일요일에 뭐하기로 했지
의도 예측 클래스 :  3
의도 예측 레이블 :  내용
